<a href="https://colab.research.google.com/github/DeepaManjunath/Assignment5-New/blob/master/Assignment5_Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe
from keras.preprocessing.image import ImageDataGenerator
import keras


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


In [0]:
tf.enable_eager_execution()

In [0]:

BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}

In [0]:

def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:

class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.drop = tf.keras.layers.Dropout(0.03555)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.drop(self.conv(inputs))))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h


In [8]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct


In [10]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS-4)], [0.050, LEARNING_RATE, 0.050])[0] 
                                  
                                 
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE

print(global_step)
print(lr_schedule)
opt = tf.train.MomentumOptimizer(lr_func,momentum=MOMENTUM,use_nesterov=False)


lr_schedule1 = lambda t: np.interp([t], [(EPOCHS-3), (EPOCHS-3), (EPOCHS)], [0.025, 0.025, 0.0])[0]

print(batches_per_epoch)
global_step1 = tf.train.get_or_create_global_step()


lr_func1=lambda: lr_schedule1(global_step1/batches_per_epoch)/BATCH_SIZE

opt1 = tf.train.MomentumOptimizer(lr_func1,momentum=MOMENTUM,use_nesterov=False)


data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)


<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=0>
<function <lambda> at 0x7f832dbe36a8>
98


In [11]:
x_train.shape

(50000, 40, 40, 3)

In [0]:
import numpy as np

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
datagen=ImageDataGenerator(      
                                                                rescale=1./255,width_shift_range=0.1,
	                                                              height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	                                                              horizontal_flip=True, fill_mode="nearest", 
                                                                rotation_range=25,preprocessing_function=get_random_eraser(v_l=0, v_h=1)   
)
	                                                             
datagen.fit(x_train)


In [28]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
 if epoch<=(EPOCHS-3):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)
  
 else:
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt1.apply_gradients(zip(grads, var), global_step=global_step1)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)
  

  

epoch: 1 lr: 0.12000000000000001 train loss: 0.40313922241210937 train acc: 0.86016 val loss: 0.5518427017211914 val acc: 0.8151 time: 58.24988341331482


epoch: 2 lr: 0.19 train loss: 0.3571139569091797 train acc: 0.87818 val loss: 0.47322737579345703 val acc: 0.8402 time: 116.67440986633301


epoch: 3 lr: 0.26 train loss: 0.3230510624694824 train acc: 0.88874 val loss: 0.40317477951049807 val acc: 0.8672 time: 174.88553047180176


epoch: 4 lr: 0.33 train loss: 0.29218284378051756 train acc: 0.89828 val loss: 0.4346499183654785 val acc: 0.8562 time: 233.53374433517456


epoch: 5 lr: 0.4 train loss: 0.2652900238037109 train acc: 0.90708 val loss: 0.3971183380126953 val acc: 0.8684 time: 292.3039689064026


...

epoch: 20 lr: 0.05 train loss: 0.09968291046142579 train acc: 0.96668 val loss: 0.33504225387573244 val acc: 0.8971 time: 1177.8770837783813


epoch: 21 lr: 0.05 train loss: 0.09494134567260742 train acc: 0.96836 val loss: 0.3078335632324219 val acc: 0.9073 time: 1236.7461576461792


epoch: 22 lr: 0.05 train loss: 0.08911432605743408 train acc: 0.97026 val loss: 0.33515089797973635 val acc: 0.9003 time: 1295.9310569763184


epoch: 23 lr: 0.05 train loss: 0.08467948959350587 train acc: 0.9725 val loss: 0.31268115463256835 val acc: 0.906 time: 1355.0404472351074


epoch: 24 lr: 0.05 train loss: 0.08192710075378418 train acc: 0.97346 val loss: 0.3125067642211914 val acc: 0.9064 time: 1414.2962543964386
